In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('DDO12_filled.csv')
df_joined = pd.read_csv('_SELECT_r_p_v_vd_FROM_record_r_JOIN_person_p_ON_r_person_index_p_202408261231.csv').drop('Unnamed: 0', axis=1)

<ipython-input-5-afc71d9c3b2c>:2: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_joined = pd.read_csv('_SELECT_r_p_v_vd_FROM_record_r_JOIN_person_p_ON_r_person_index_p_202408261231.csv').drop('Unnamed: 0', axis=1)


In [24]:
def extract_details(text):
    # Patterns for embarkation
    embark_patterns = {
        'embarqué': {                                                                                           
            'date': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées|resté)\b).)*le (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) à ([\w\s\'-îÎ]+?)(?=\s+\b(?:mort[es]?|débarqué[es]?|déserté[es]?|passé[es]?|resté[es]?|tombé[es]?)\b| le \d{2}/\d{2}/\d{4}|,| ---|\n|$)'
        },
        'a fait la': {
            'date': r'remplacement du (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de ([\w\s\'-îÎ]+?) à ([\w\s\'-îÎ]+)'
        },
        'supplément': {
            'date': r'\bsupplément à (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? du (\d{2}/\d{2}/\d{4})',
            'location': r'\bsupplément à ([\w\s\'-îÎ]+?) du \d{2}/\d{2}/\d{4}'
        },
        'remplacement': {
            'date': r'\bremplacement (?:au|à)?(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+?(?: le| du) (\d{2}/\d{2}/\d{4})',
            'location': r'\bremplacement (?:au|à)? ([\w\s\'-îÎ]+?)(?: en| le| du|\n|,| ---|$)'
        },
        'trouvé': {
            'date': r'\btrouvé[es]? caché[es]? à bord(?:(?!\b(?:débarqué[es]?|déserté[es]?|mort[es]?|passé[es]?)\b)[^\n])*? le (\d{2}/\d{2}/\d{4})',
            'location': r'\btrouvé[es]? caché[es]? à bord (?:après le départ de|le \d{2}/\d{2}/\d{4})\s+((?!\b(?:débarqué[es]?|sert|déserté[es]?|mort[es]?|passé[es]?)\b)[\w\s\'-îÎ]+?)(?=\s+\b(?:le\s+\d{2}/\d{2}/\d{4}|---|,|\n|$|\b(?:débarqué[es]?|sert|déserté[es]?|mort[es]?|passé[es]?)\b))'
        }
    }

    # Patterns for disembarkation
    disembark_patterns = {
        'débarqué': {
            'date': r'\bdébarqué[es]?.*?(?:à|au|furtivement à|malade après être tombé du haut mal à)? (?:désarmement à |malade à |malade et mort à l\'hôpital (?:de|du) |malade à l\'hôpital (?:de|du) )?[\w\s\'-îÎ]+? le (\d{2}/\d{2}/\d{4})',
            'location': r'\bdébarqué[es]? (?:au |à |furtivement à |malade après être tombé du haut mal à )?(?:désarmement à |malade à l\'hôpital (?:de|du) |malade à |malade et mort à l\'hôpital (?:de|du) )?([\w\s\'-îÎ]+?)(?= le \d{2}/\d{2}/\d{4}| ---|,|\n|$| mort[es]?)'
        },
        'déserté': {
            'date': r'\bdéserté[es]? (?:à [\w\s\'-îÎ]+|sur le vaisseau de côte le [\w\s\'-îÎ]+|en [\w\s\'-îÎ]+) le (\d{2}/\d{2}/\d{4})',
            'location': r'\bdéserté[es]? (?:à|en|au départ de) ([\w\s\'-îÎ]+?)(?: le|\n|,| ---|$)'
        },
        'mort en mer': {
            'date': r'\b(?:mort[es]? (?:en mer|noyé en [\w\s\'-îÎ]+|du [\w\s\'-îÎ]+ en [\w\s\'-îÎ]+|à l\'hôpital de [\w\s\'-îÎ]+|à la ration|à [\w\s\'-îÎ]+)|tombé à la mer et mort noyé) le (\d{2}/\d{2}/\d{4})',
            'location': r'\bmort[es]? (?:en mer|à l\'hôpital (?:du|de)|à la ration|à) ([\w\s\'-îÎ]+?)(?= le \d{2}/\d{2}/\d{4})'
        },
        'a fait la': {
            'date': r'levé[es]? du (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de [\w\s\'-îÎ]+? à ([\w\s\'-îÎ]+?)(?=\s*(?:du|le) \d{2}/\d{2}/\d{4}|\s*---|,|en|\n|$)'
        },
        'passé': {
            'date': r'\bpassé[es]? sur la [\w\s\'-îÎ]+ en rade de [\w\s\'-îÎ]+ le (\d{2}/\d{2}/\d{4})',
            'location': r'\bpassé[es]? sur la [\w\s\'-îÎ]+ en rade de ([\w\s\'-îÎ]+)(?=\s+(?:le\s+\d{2}/\d{2}/\d{4}|,| ---|\n|$))'
        },
        'resté': {
            'date': r'\bresté[es]? (?:malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de|à l\'hôpital de) [\w\s\'-îÎ]+? le\s*(\d{2}/\d{2}/\d{4})(?: rejoint|\n|,| ---|$)',
             'location': r'\bresté[es]? (?:malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de|à l\'hôpital de) ([\w\s\'-îÎ]+?)(?: le| rejoint|\n|,| ---|$)'

        }
    }

    # Initialize results
    embark_location = pd.NA
    embark_date = pd.NA
    disembark_location = pd.NA
    disembark_date = pd.NA

    # Initialize match variables
    embark_date_match = None
    embark_location_match = None
    disembark_date_match = None
    disembark_location_match = None

    # Determine which embark patterns to use
    if 'embarqué' in text.lower() or 'rembarqué' in text.lower():
        embark_date_match = re.search(embark_patterns['embarqué']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['embarqué']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        embark_date_match = re.search(embark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'supplément' in text.lower():
        embark_date_match = re.search(embark_patterns['supplément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['supplément']['location'], text, flags=re.IGNORECASE)
    elif 'remplacement' in text.lower():
        embark_date_match = re.search(embark_patterns['remplacement']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['remplacement']['location'], text, flags=re.IGNORECASE)
    elif 'trouvé' in text.lower():
        embark_date_match = re.search(embark_patterns['trouvé']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['trouvé']['location'], text, flags=re.IGNORECASE)        

    # Extract embark information
    if embark_date_match:
        embark_date = embark_date_match.group(1)
    if embark_location_match:
        embark_location = embark_location_match.group(1)

    # Determine which disembark patterns to use
    if 'débarqué' in text.lower():
        disembark_date_match = re.search(disembark_patterns['débarqué']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['débarqué']['location'], text, flags=re.IGNORECASE)
    elif 'déserté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['déserté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['déserté']['location'], text, flags=re.IGNORECASE)
    elif 'mort' in text.lower():
        disembark_date_match = re.search(disembark_patterns['mort en mer']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['mort en mer']['location'], text, flags=re.IGNORECASE)
    elif 'passé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['passé']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['passé']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        disembark_date_match = re.search(disembark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'resté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['resté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['resté']['location'], text, flags=re.IGNORECASE)

    # Extract disembark information
    if disembark_date_match:
        disembark_date = disembark_date_match.group(1)
    if disembark_location_match:
        disembark_location = disembark_location_match.group(1)
        if len(disembark_location_match.groups()) > 1 and disembark_location_match.group(2):
            disembark_location += f", {disembark_location_match.group(2)}"
    if 'mort à bord' in text.lower() or 'morte à bord' in text.lower() or 'morts à bord' in text.lower() or 'mortes à bord' in text.lower():
        disembark_location = 'on board'
    elif 'mort en mer' in text.lower() or 'morte en mer' in text.lower() or 'morts en mer' in text.lower() or 'mortes en mer' in text.lower():
        disembark_location = 'at sea'
    if 'née en mer' in text.lower() or 'né en mer' in text.lower() or 'nées en mer' in text.lower() or 'nés en mer' in text.lower():
        embark_location = 'at sea'
    elif 'née à bord' in text.lower() or 'né à bord' in text.lower() or 'nées à bord' in text.lower() or 'nés à bord' in text.lower():
        embark_location = 'on board'
    
            

    return embark_location, embark_date, disembark_location, disembark_date

In [26]:
df[df.Disemb_loc=='désarmement']

,Unnamed: 0,n,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,Emb_loc,emb_class,disemb_class,issued
61,61,161.0,BLAINO,Jean,NaN,rembarqué à bord La Villeflix \n revenu à bord...,21.0,15,01/01/1755,20/12/1753,désarmement,bord La Villeflix revenu à bord,306.0,302,[]
748,748,620.0,TRUSCAT,Jérôme,NaN,rembarqué au départ du La Villeflix \n débarqu...,NaN,15,NaN,22/06/1755,désarmement,NaN,306.0,302,"['emb_date', 'emb_loc']"


In [17]:
df_joined[(df_joined.last_name=='VALENTIN') & (df_joined.first_name=='Antoine')].iloc[0]['steps'].split("-")[-1]

' Sénégal'

In [23]:
df_joined

,remarks,emb_date,emb_loc,emb_class,disemb_date,disemb_loc,disemb_class,last_name,first_name,date_voy_begin,date_voy_end,steps,start_loc,last_loc,place
0,a fait la campagne,21/03/1722,Lorient,301.0,16/12/1722,Lorient,301.0,BOUEDELAN,Pierre,21/03/1722,16/12/1722,Lorient - au devant des vaisseaux - Lorient,Lorient,Lorient,Port-Louis
1,a fait la campagne,21/03/1722,Lorient,301.0,16/12/1722,Lorient,301.0,ALLÉ,Joseph,21/03/1722,16/12/1722,Lorient - au devant des vaisseaux - Lorient,Lorient,Lorient,Port-Louis
2,a fait la campagne,21/03/1722,Lorient,301.0,16/12/1722,Lorient,301.0,LEGER,Charles,21/03/1722,16/12/1722,Lorient - au devant des vaisseaux - Lorient,Lorient,Lorient,Port-Louis
3,a fait la campagne,21/03/1722,Lorient,301.0,16/12/1722,Lorient,301.0,BOUEZAR,Étienne,21/03/1722,16/12/1722,Lorient - au devant des vaisseaux - Lorient,Lorient,Lorient,Port-Louis
4,a fait la campagne,21/03/1722,Lorient,301.0,16/12/1722,Lorient,301.0,BASSET,Guillaume,21/03/1722,16/12/1722,Lorient - au devant des vaisseaux - Lorient,Lorient,Lorient,Port-Louis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210224,embarqué à l'île de France ou à l'île Bourbon ...,NaN,NaN,NaN,08/02/1757,Lorient,301.0,MOUTON,Jacques,28/11/1754,08/02/1757,Lorient - Cap-Vert - Cap de Bonne-Espérance - ...,Lorient,Lorient,"Santiago, Cap-Vert"
210225,embarqué à l'île de France ou à l'île Bourbon ...,NaN,NaN,NaN,08/02/1757,Lorient,301.0,GRIGNON,NaN,28/11/1754,08/02/1757,Lorient - Cap-Vert - Cap de Bonne-Espérance - ...,Lorient,Lorient,"Santiago, Cap-Vert"
210226,embarqué à l'île de France ou à l'île Bourbon ...,NaN,NaN,NaN,08/02/1757,Lorient,301.0,DUPLESSY,NaN,28/11/1754,08/02/1757,Lorient - Cap-Vert - Cap de Bonne-Espérance - ...,Lorient,Lorient,"Santiago, Cap-Vert"
210227,embarqué à l'île de France ou à l'île Bourbon ...,NaN,NaN,NaN,08/02/1757,Lorient,301.0,LANGLIER,NaN,28/11/1754,08/02/1757,Lorient - Cap-Vert - Cap de Bonne-Espérance - ...,Lorient,Lorient,"Santiago, Cap-Vert"
